Skoro wiemy już czym jest i jak działa model regresji liniowej warto się zastanowić na co powinnśmy zwrócić uwagę podczas modelowania (zasady te będą równie ważne podaczas używania innych modeli niż liniowy).

# Współliniowość zmiennych

Budując model powinniśmy zwrócić uwagę na to czy nasze predyktory nie są współliniowe (jeden z predyktorów jest kombinacją liniową innych). Wprowadzanie współliniowych predyktorów powoduje przekazywanie identycznej informacji z wielu miejsc:

In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
data = pd.read_csv("data/colinearity.csv")
data

,x1,x2,z,y
0,-0.032436,3.302115,12.839714,-2.658624
1,0.037800,2.945636,10.393777,-2.087332
2,0.192258,2.132955,7.549086,-1.072105
3,0.554356,3.454952,10.683218,-1.375270
4,0.610369,3.591809,11.080006,-2.095570
...,...,...,...,...
95,9.417322,8.297676,7.269898,13.760754
96,9.977041,6.831405,-0.545786,14.007908
97,9.534817,9.717499,11.455468,13.424037
98,9.849744,8.195875,5.435839,15.623582


In [2]:
X1 = sm.add_constant(data[['x1', 'x2']].values)
X2 = sm.add_constant(data[['x1', 'x2', 'z']].values)
y = data['y'].values
reg1 = sm.OLS(y, X1).fit()
reg2 = sm.OLS(y, X2).fit()

In [3]:
print("Coefficients:\n", reg1.params)
print("\nP-Values:\n", reg1.pvalues)
print("\nSignificant:\n", reg1.pvalues < 0.01)

Coefficients:
 [-0.12972163  2.14580211 -0.80335749]

P-Values:
 [5.37085479e-01 3.47026603e-69 3.97607766e-28]

Significant:
 [False  True  True]


In [4]:
print("Coefficients:\n", reg2.params)
print("\nP-Values:\n", reg2.pvalues)
print("\nSignificant:\n", reg2.pvalues < 0.01)

Coefficients:
 [-0.58649516  3.23996238 -2.35307049  0.46668154]

P-Values:
 [3.49597403e-02 6.64077853e-11 2.78028676e-04 1.43524745e-02]

Significant:
 [False  True  True False]


Objawy współliniowosci:

 - Duże zmiany współczynników regresji pod dodaniu/odjęciu zmiennej.
 - Wysoka wartość predykcyjna modelu (np wysokie $R^2$) lecz niesistotne zmienne.

Najprostszym sposobem badania współliniowości predyktorów jest policzenie korelacji:

In [5]:
corrx1x2 = np.corrcoef(data.x1, data.x2)[0, 1]
print("Correlation coefficient between x1 and x2:", corrx1x2)
corrx1z = np.corrcoef(data.x1, data.z)[0, 1]
print("Correlation coefficient between x1 and z:", corrx1z)
corrx2z = np.corrcoef(data.x2, data.z)[0, 1]
print("Correlation coefficient between x2 and z:", corrx2z)

Correlation coefficient between x1 and x2: 0.5951334425069779
Correlation coefficient between x1 and z: -0.2582590518358908
Correlation coefficient between x2 and z: 0.6201384259726195


Jeśli widzimy 2 mocno skorelowane ze sobą predyktory to powinnismy pozbyć się jednego z nich. Niestety ten sposób nie działa gdy jeden predyktor jest kombinacją pozostałych.

In [6]:
corr_lin_comb = np.corrcoef(-2.37*data.x1 + 3.33*data.x2 + 1, data.z)[0, 1]
print("Correlation coefficient between -2.37*x1 + 3.33*x2 + 1 and z:", corr_lin_comb)

Correlation coefficient between -2.37*x1 + 3.33*x2 + 1 and z: 0.9969336749167262


Innym sposobem jest policzenie współczynnika VIF (Variance inflation factor). W skrócie dla danego predyktora tworzymy model liniowy zakładając zależność od pzostałych predyktórw. Nastepnie liczymy VIF jako:

$$
1 / (1 - R^2)
$$

Jeśli VIF jest wysoki (w literaturze 5+) mamy problem ze współliniowością zmiennych.

In [7]:
X1 = sm.add_constant(data[['x1', 'x2']].values)
z = data['z'].values
reg3 = sm.OLS(z, X1).fit()
print("Coefficients:\n", reg3.params)
print("\nP-Values:\n", reg3.pvalues)
print("\nSignificant:\n", reg3.pvalues < 0.01)
VIF_z = 1 / (1 - reg3.rsquared)
print("\nVIF for z variable:\n", VIF_z)

Coefficients:
 [ 0.97876922 -2.34455441  3.32070772]

P-Values:
 [4.21102823e-014 4.05131249e-099 6.85332409e-108]

Significant:
 [ True  True  True]

VIF for z variable:
 164.83453929475615


Ostatnim sposobem jest Test Farrar'a–Glauber'a, którego hipotezą zerową jest brak współliniowości w modelu:

In [8]:
from statsmodels.stats.diagnostic import het_breuschpagan
fg_test = het_breuschpagan(reg2.resid, X2)
print(f"Breusch-Pagan test statistic: {fg_test[0]:.4f}")
print(f"p-value: {fg_test[1]:.4f}")

Breusch-Pagan test statistic: 5.8459
p-value: 0.1194
